In [1]:
#export
import k1lib as k1, os
import k1lib.cli as cli
from collections import deque
from functools import lru_cache
from contextlib import contextmanager
__all__ = ["sql", "s3", "s3bucket"]

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#export
@contextmanager
def mysqlCnf(user, password, host, port):
    fn = f"""[client]\nuser = "{user}"\npassword = "{password or ''}"\nhost = "{host}"\nport = "{port}" """ | cli.file()
    try: yield fn
    finally: os.remove(fn)
mysqlConn = k1.dep("mysql.connector", "mysql-connector-python", "https://pypi.org/project/mysql-connector-python/")
pgConn = k1.dep("psycopg2", "psycopg2-binary", "https://pypi.org/project/psycopg2/")
sqlite3 = k1.dep("sqlite3")
qD = {"my": "`", "pg": "", "lite": ""} # quote dict
class sql:
    def __init__(self, host, port=3306, user=None, password="", mode="my"):
        """Creates a connection to a SQL database.
Example::

    s = sql("localhost") # creates a new sql object
    s.refresh()          # refreshes connection any time you encounter strange bugs
    
    s | ls()                                                       # lists out all databases
    s | toBytes()                                                  # dumps every databases, returns Iterator[str]. Yes, it's "toBytes", but it has the feeling of serializing whatever the input is, so it's at least intuitive in that way
    "dump.sql" | s                                                 # restores the database using the dump file
    cat("dump.sql") | s                                            # restores the database using the dump file
    db1 = s | ls() | filt(lambda x: x.name == "db1") | item()      # gets database named "db1"
    db1 = s | ls() | grep("db1") | item()                          # gets database with name that contains "db1" string, which is more convenient (could get "db1" or "db1_a" or "a_db1" btw, so be careful)
    db1 | ls()                                                     # list out all tables within this database
    db1 | toBytes()                                                # dumps the database, returns Iterator[str]
    users = db1 | ls() | filt(lambda x: x.name == "user") | item() # gets table named "user"
    users = db1 | ls() | grep("user") | item()                     # gets table with name that contains "user", which is more convenient (could get "user" or "user_vehicle", so again, be careful)
    
    db1.query("select * from users")                     # queries the database using your custom query
    db1.query("select * from users where user_id=%s", 3) # queries with prepared statement
    
    users.info()                                               # prints out the first 10 rows of the table and the table schema
    users.cols                                                 # returns table's columns as List[str]
    users.query(...)                                           # can also do a custom query, just like with databases
    users | toBytes()                                          # dumps this specific table, returns Iterator[str]
    users |  cat() | display()                                 # reads entire table, gets first 10 rows and displays it out
    users | (cat() | head(20)) | display()                     # reads first 20 rows only, then displays the first 10 rows. Query sent is "select * from user limit 20"
    users | (cat() | filt("x == 4", 3) | head(20)) | display() # grabs first 20 rows that has the 4th column equal to 4, then displays the first 10 rows. Query sent is "select user_id, address, balance, age from user where age = 4", assuming the table only has those columns

:param host: host name, ip address, or file name (in case of sqlite)
:param port: port at the host
:param uesr: database user name. If not specified then fallback to environment variable ``SQL_USER``, then ``USER``
:param password: database password. If not specified then assume database doesn't require one
:param mode: currently supports 3 values: "my" (MySQL), "pg" (PostgreSQL) and "lite" (SQLite)
"""
        self.host = host; self.port = port; self.user = user or os.environ.get("SQL_USER") or os.environ.get("USER")
        self.password = password or os.environ.get("SQL_PASSWORD"); self.db = None; self.mode = mode; self.conn = None; self.refresh()
    def refresh(self):
        try: self.conn.close(); self.conn = None
        except: pass
        kwargs = dict(host=self.host, port=self.port, user=self.user, password=self.password, database=self.db)
        if self.mode == "my": self.conn = mysqlConn.connect(**kwargs, charset='utf8mb4', collation='utf8mb4_general_ci')
        elif self.mode == "pg": self.conn = pgConn.connect(**kwargs)
        elif self.mode == "lite": self.conn = sqlite3.connect(self.host)
        else: raise Exception(f"Can only support 'my' (MySQL), 'pg' (PostgreSQL) and 'lite' (SQLite) for now, can't support {self.mode}")
    def _changeDb(self, db):
        if self.db != db:
            if self.mode == "my": self.query(f"use `{db}`")
            elif self.mode == "pg": self.db = db; self.refresh()
            elif self.mode == "lite": pass
        self.db = db
    def query(self, query, *args):
        cur = self.conn.cursor(); cur.execute(query, args)
        try: ans = cur.fetchall()
        except: ans = None
        cur.close(); self.conn.commit(); return ans
    def _ls(self):
        if self.mode == "my": return [sqldb(self, e[0]) for e in self.query("show databases")]
        elif self.mode == "pg": return [sqldb(self, e[0]) for e in self.query("select datname from pg_database where datistemplate=false")]
        elif self.mode == "lite": return [sqldb(self, "default")]
    def __repr__(self): return f"<sql mode={self.mode} host={self._host}>"
    @property
    def _host(self) -> str:
        if self.mode == "lite": return self.host.split(os.sep)[-1]
        else: return f"{self.host}:{self.port}"
    def _cnfCtx(self): return mysqlCnf(self.user, self.password, self.host, self.port)
    def _toBytes(self):
        if self.mode == "my":
            with self._cnfCtx() as fn: yield from None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob --all-databases")
        else: raise Exception(f"All databases dump of mode {self.mode} is not supported yet")
    def __ror__(self, it): # restoring a backup
        if self.mode == "my":
            def restore(fn):
                with self._cnfCtx() as cnfFn: None | cli.cmd(f"mysql --defaults-file={cnfFn} < {fn}") | cli.ignore()
            if isinstance(it, str):     restore(it)
            else: fn = it | cli.file(); restore(fn); os.remove(fn)
        else: raise Exception(f"Restoring database from .sql file of mode {self.mode} is not supported yet")
class sqldb:
    def __init__(self, sql:sql, name:str):
        """A sql database representation. Not expected to be instatiated by you. See also: :class:`sql`"""
        self.sql = sql; self.name = name
    def query(self, query, *args): self.sql._changeDb(self.name); return self.sql.query(query, *args)
    def _ls(self):
        if self.sql.mode == "my": return [sqltable(self.sql, self, e[0]) for e in self.query(f"show tables")]
        if self.sql.mode == "pg": return [sqltable(self.sql, self, e[0]) for e in self.query(f"select table_name from information_schema.tables")]
        if self.sql.mode == "lite": return [sqltable(self.sql, self, e[0]) for e in self.query("select name from sqlite_master where type='table'")]
    def __repr__(self): return f"<sqldb host={self.sql._host} db={self.name}>"
    def _toBytes(self):
        if self.sql.mode == "my":
            with self.sql._cnfCtx() as fn: yield from None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob --databases {self.name}")
        else: raise Exception(f"Database dump of mode {self.sql.mode} is not supported yet")
    def __ror__(self, it): return self.sql.__ror__(it)
class sqltable:
    def __init__(self, sql, sqldb, name:str):
        """A sql table representation. Not expected to be instantiated by you. See also: :class:`sql`"""
        self.sql = sql; self.sqldb = sqldb; self.name = name; self._cols = None
    def _cat(self, ser):
        cols = self.cols; _2 = [] # clis that can't be optimized, stashed away to be merged with ser later on
        q = qD[self.sql.mode]; clis = deque(ser.clis)
        o1 = None # cut() opt
        o2 = None # head() opt
        o3 = [] # filt() opt
        while len(clis) > 0:
            c = clis.popleft()
            if isinstance(c, cli.filt): _2.append(c); break # TODO: add optimizations for filt
            elif o2 is None and isinstance(c, cli.head):
                if round(c.n) != c.n or c.n < 0 or c.inverted or c.n == None: _2.append(c); break
                else: o2 = f"limit {c.n}"; continue
            elif o1 is None and isinstance(c, cli.cut):
                if isinstance(c.columns, slice): _2.append(c); o1 = 0; continue
                else:
                    o1 = ", ".join([f"{q}{c}{q}" for c in cols | cli.rows(*c.columns)])
                    if len(c.columns) == 1: _2.append(cli.item().all() | cli.aS(list))
            else: _2.append(c); break
        o1 = o1 or ", ".join([f"{q}{c}{q}" for c in cols])
        query = f"select {o1} from {q}{self.name}{q} {o2 or ''}"#; print(f"query: {query}"); return []
        return self.sqldb.query(query) | cli.serial(*_2, *clis)
    @property
    def cols(self):
        """Get column names"""
        if not self._cols: self._cols = self.describe()[1:] | cli.cut({"my": 0, "pg": 0, "lite": 1}[self.sql.mode]) | cli.deref()
        return self._cols
    @lru_cache
    def describe(self):
        if self.sql.mode == "my": return self.sqldb.query(f"describe `{self.name}`") | cli.insert(["Field", "Type", "Null", "Key", "Default", "Extra"]) | cli.deref()
        if self.sql.mode == "pg": return self.sqldb.query(f"select column_name, data_type, is_nullable, column_default, ordinal_position from information_schema.columns where table_name='{self.name}'") | cli.insert(["column_name", "data_type", "is_nullable", "column_default", "ordinal_position"]) | cli.deref()
        if self.sql.mode == "lite": return self.sqldb.query(f"pragma table_info([{self.name}])") | cli.insert(["cid", "name", "type", "notnull", "dflt_value", "pk"]) | cli.deref()
    def info(self, out=False):
        """Preview table

:param out: if True, returns a list of lines instead of printing them out"""
        def gen():
            print(f"Table `{self.name}`\n")
            desc = self.describe() | cli.deref(); cols = self.cols; q = qD[self.sql.mode]; s = ", ".join([f"{q}{e}{q}" for e in cols])
            self.sqldb.query(f"select {s} from {q}{self.name}{q} limit 9") | (cli.aS(repr) | cli.head(50)).all(2) | cli.insert(cols) | cli.display(); print("")
            desc | cli.display(None)
        if out:
            with k1.captureStdout() as out: gen()
            return out()
        else: gen()
    def query(self, query, *args): return self.sqldb.query(query, *args)
    def __repr__(self): return f"<sqltable host={self.sql._host} db={self.sqldb.name} table={self.name}>"
    def _toBytes(self):
        if self.sql.mode == "my":
            with self.sql._cnfCtx() as fn: yield from None | cli.cmd(f"mysqldump --defaults-file={fn} --single-transaction --hex-blob {self.sqldb.name} {self.name}")
        else: raise Exception(f"Table dump of mode {self.sql.mode} is not supported yet")
    def __ror__(self, it): return self.sql.__ror__(it)

In [3]:
s = sql("localhost"); s.refresh()
assert s | cli.ls() | cli.shape(0) > 0
assert s | cli.ls() | cli.filt("x.name == 'truckbux'") | cli.shape(0) > 0
db1 = s | cli.ls() | cli.grep("truckbux") | cli.item()
t1 = db1 | cli.ls() | cli.grep("noti") | cli.item(); t1.info(); t1.cols
a = t1.query("select id from notification") | cli.shape(0)
b = t1 | cli.cat() | cli.shape(0)
assert a == b; assert a > 1000

Table `notification`

notification_id   id   app_type   notification_description                             create_date                                  is_visited   is_active   
1                 2    'USER'     b'Nice! Your order TB-78593DF has successfully bee   datetime.datetime(2017, 10, 19, 4, 34, 14)   1            1           
2                 2    'USER'     b'Nice! Your order TB-4EC736F has successfully bee   datetime.datetime(2017, 10, 19, 4, 35, 20)   1            1           
3                 2    'USER'     b'Congrats! Your order TB-78593DF has been accepte   datetime.datetime(2017, 10, 19, 4, 35, 52)   1            1           
4                 2    'USER'     b'Ayee! Your order TB-78593DF is ready to go. Come   datetime.datetime(2017, 10, 19, 4, 35, 58)   1            1           
5                 2    'USER'     b'Order TB-78593DF was picked up at 12:36am. Enjoy   datetime.datetime(2017, 10, 19, 4, 36, 12)   1            1           
6                 1    'VENDOR

In [4]:
s = sql("localhost", 5432, "postgres", "postgres", mode="pg")
t = s | cli.ls() | cli.item() | cli.ls() | cli.item()
assert t.query("select * from pg_statistic") | cli.shape(0) > 0
t.info(); t | (cli.cat() | cli.head()) | cli.display()

Table `pg_statistic`

starelid   staattnum   stainherit   stanullfrac   stawidth   stadistinct   stakind1   stakind2   stakind3   stakind4   stakind5   staop1   staop2   staop3   staop4   staop5   stacoll1   stacoll2   stacoll3   stacoll4   stacoll5   stanumbers1                                          stanumbers2    stanumbers3      stanumbers4   stanumbers5   stavalues1                                           stavalues2                                           stavalues3   stavalues4   stavalues5   
1247       1           False        0.0           4          -1.0          2          3          0          0          0          609      609      0        0        0        0          0          0          0          0          None                                                 [0.9784336]    None             None          None          '{16,22,28,81,199,273,604,701,790,1002,1009,1015,1   None                                                 None         None         None         


In [5]:
s = sql("/home/kelvin/ssd/data/wikidata/wikidata.db", mode="lite")
t = s | cli.ls() | cli.item() | cli.ls() | cli.item()
assert t.query("select * from wikidata limit 10") | cli.shape(0) == 10
t.info(); t | (cli.cat() | cli.head()) | cli.display()

Table `wikidata`

idx   type     id        pageid   ns   title     lastrevid    label                 desc                                                 
0     'item'   'Q31'     127      0    'Q31'     1713980862   'Belgium'             'country in western Europe since 1830'               
1     'item'   'Q8'      134      0    'Q8'      1717439938   'happiness'           'mental or emotional state of well-being character   
2     'item'   'Q23'     136      0    'Q23'     1721165431   'George Washington'   'president of the United States from 1789 to 1797'   
3     'item'   'Q24'     137      0    'Q24'     1713446920   'Jack Bauer'          'character from the television series 24'            
4     'item'   'Q42'     138      0    'Q42'     1717100210   'Douglas Adams'       'English science fiction writer and humourist (195   
5     'item'   'Q1868'   142      0    'Q1868'   1710866440   'Paul Otlet'          'Belgian author, librarian and colonial thinker'     
6     'item'   '

In [6]:
from k1lib.imports import *
s = sql("/home/kelvin/ssd/data/wikidata/wikidata.db", mode="lite") | ls() | item(); s | ls()

2023-11-02 11:40:55,321	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-11-02 11:40:55,328	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


[<sqltable host=wikidata.db db=default table=wikidata>,
 <sqltable host=wikidata.db db=default table=descIndex>,
 <sqltable host=wikidata.db db=default table=labelIndex>,
 <sqltable host=wikidata.db db=default table=subDescIndex>,
 <sqltable host=wikidata.db db=default table=subLabelIndex>]

In [7]:
s | ls() | apply(op().info(True) | (item() | aS(fmt.h, 3)) & (~head(2) | join("\n") | aS(fmt.pre)) | join("\n")) | join("\n") | aS(IPython.display.HTML)

In [8]:
#export
boto3 = k1.dep("boto3")
class s3:
    def __init__(self, client):
        """Represents an S3 client.
Example::

    client = boto3.client("s3", ...)            # put your credentials and details here
    db = s3(client)                             # creates an S3 manager
    db | ls()                                   # lists all buckets accessible

    bucket = db | ls() | item()                 # grabs the first bucket, returns object of type s3bucket
    bucket = s3bucket(client, "bucket-name")    # or you can instantiate the bucket directly
    bucket | ls()                               # lists all objects within this bucket
    bucket | ls() | grep("\\.so")               # grabs all .so files from the bucket

    obj = bucket | ls() | item()                # grabs the first object within this bucket, returns object of type s3obj
    obj.key, obj.size, obj.lastModified         # some fields directly accessible

This mostly offers interoperability with ls() and cat(), so that you can
write relatively intuitive code, but fundamentally provides no upsides"""
        self.client = client
    def _ls(self): return [s3bucket(self.client, x["Name"]) for x in self.client.list_buckets()["Buckets"]]
    def __repr__(self): return f"<kaws.s3 client>"
class s3bucket:
    def __init__(self, client, name:str):
        """Represents an S3 bucket.
See also: :class:`s3`"""
        self.client = client; self.name = name
    def _ls(self): client = self.client; name = self.name; return [s3obj(client, name, data) for data in self.client.list_objects(Bucket=name).get("Contents", [])]
    def __repr__(self): return f"<s3bucket name='{self.name}'>"
class s3obj:
    def __init__(self, client, bucket:str, data):
        """Represents an S3 object. Not intended to be instantiated directly.
See also: :class:`s3`"""
        self.client = client; self.bucket = bucket
        self.key  = data["Key"];  self.lastModified = data["LastModified"]
        self.size = data["Size"]; self.storageClass = data["StorageClass"]
    def __repr__(self): return f"<s3obj bucket='{self.bucket}' key='{self.key}' size={fmt.size(self.size)}>"
    def _cat(self, kwargs):
        if kwargs["text"] is not False: raise Exception(f"s3obj does not support `cat(text=True)`. Please use `cat(text=False)` instead")
        if kwargs["chunks"]: raise Exception(f"s3obj does not support `cat(chunks=True)`")
        sB = kwargs["sB"]; eB = kwargs["eB"]
        if eB < 0: eB = self.size
        return self.client.get_object(Bucket=self.bucket, Key=self.key, Range=f'bytes={sB}-{eB-1}')["Body"].read()

In [ ]:
#notest
db = s3(boto3.client('s3', aws_access_key_id=k1, aws_secret_access_key=k2))
db | ls() | item() | ls()# | grep("csv") | item() | cat(text=False, eB=100)

[<s3obj bucket='157239n-bucket-1' key='Node Network-data-2023-10-31 10_44_21.csv' size=3.27 kB>,
 <s3obj bucket='157239n-bucket-1' key='Node Network-data-as-joinbyfield-2023-10-31 10_46_48.csv' size=10.19 kB>,
 <s3obj bucket='157239n-bucket-1' key='Slot 1 x-data-as-seriestocolumns-2023-10-31 10_59_53.csv' size=3.0 B>,
 <s3obj bucket='157239n-bucket-1' key='dashboard.html' size=1.42 MB>,
 <s3obj bucket='157239n-bucket-1' key='notes.md' size=465.0 B>]

In [11]:
!../../export.py cli/lsext

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-02 11:44:11,283	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-11-02 11:44:11,289	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
7596    1   40%   
11437   0   60%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.2
Uninstalling k1lib-1.4.4.2:
  Successfully uninstalled k1lib-1.4.4.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-p